In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

# ==========================================
# 1. CONFIGURACIÓN Y CARGA DE DATOS
# ==========================================
filename = "Excel.xlsx"
AMIGOS_A_JUNTAR = ["SANCHEZ", "LACOUTURE"] # Nombres clave para intentar juntar

# Carga segura
if os.path.exists(filename):
    try:
        df = pd.read_excel(filename, engine='openpyxl')
    except:
        df = pd.read_csv("Excel.xlsx - Hoja1.csv")
else:
    df = pd.DataFrame(columns=["Nombres", "A", "B", "C", "D"])

# --- SIMULACIÓN (Si el archivo está vacío/ceros) ---
# Generamos datos de 0 a 10 si detectamos que todo es 0
if len(df) > 0 and df[['A', 'B', 'C', 'D']].sum().sum() == 0:
    print("⚠️ Datos en 0 detectados. Simulando puntajes aleatorios (0-10)...")
    np.random.seed(42)
    for col in ['A', 'B', 'C', 'D']:
        df[col] = np.random.randint(0, 11, len(df))

# ==========================================
# 2. PREPROCESAMIENTO INTELIGENTE
# ==========================================
features = ['A', 'B', 'C', 'D']

# Normalizamos para saber quién destaca más RELATIVAMENTE (0 a 1)
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)

df['Rol_Principal'] = df_scaled.idxmax(axis=1)
df['Puntaje_Normalizado'] = df_scaled.max(axis=1) # Usamos esto para medir "fuerza" global

# Ordenamos por los mejores para asignarlos primero (Greedy Strategy)
df_sorted = df.sort_values(by='Puntaje_Normalizado', ascending=False).copy()

# ==========================================
# 3. ALGORITMO DE GRUPOS EQUITATIVOS
# ==========================================

def calcular_poder_grupo(grupo):
    """Suma los puntajes normalizados de los integrantes actuales del grupo"""
    suma = 0
    for integrante in grupo.values():
        if integrante is not None:
            suma += integrante['Puntaje_Normalizado']
    return suma

def crear_grupos_equitativos(df_input, amigos_prioritarios=[]):
    pendientes = df_input.to_dict('records')
    num_grupos = len(pendientes) // 4
    
    # Inicializamos grupos
    grupos = [{k: None for k in features} for _ in range(num_grupos)]
    
    # --- FASE 0: Prioridad Amigos (Opcional) ---
    vips = []
    for nombre in amigos_prioritarios:
        encontrados = [p for p in pendientes if nombre.lower() in p['Nombres'].lower()]
        if encontrados:
            vips.append(encontrados[0])
            pendientes.remove(encontrados[0])
    
    # Intentar poner a los amigos en el grupo con menos gente (o el primero vacío)
    if len(vips) >= 2:
        # Elegimos el primer grupo para los amigos
        grupo_vip = grupos[0] 
        amigo1, amigo2 = vips[0], vips[1]
        
        # Buscamos roles compatibles
        roles1 = sorted(features, key=lambda k: amigo1[k], reverse=True)
        roles2 = sorted(features, key=lambda k: amigo2[k], reverse=True)
        
        asignados = False
        for r1 in roles1:
            for r2 in roles2:
                if r1 != r2: # Roles distintos
                    grupo_vip[r1] = amigo1
                    grupo_vip[r2] = amigo2
                    amigo1['Rol_Final'] = r1
                    amigo2['Rol_Final'] = r2
                    amigo1['Nota'] = "🤝 Amigo"
                    amigo2['Nota'] = "🤝 Amigo"
                    asignados = True
                    break
            if asignados: break
        
        # Si no se pudo (mismo rol fuerte), regresan al pool
        if not asignados:
            pendientes.extend(vips)
    elif len(vips) == 1: # Si solo encontró uno, lo devuelve
        pendientes.extend(vips)

    # --- FASE 1: Asignación Equitativa ---
    sin_asignar = []
    
    for estudiante in pendientes:
        rol = estudiante['Rol_Principal']
        
        # 1. Buscar grupos que tengan este rol VACÍO
        grupos_candidatos = [g for g in grupos if g[rol] is None]
        
        if grupos_candidatos:
            # 2. ESTRATEGIA DE EQUIDAD:
            # De los candidatos, elegir el que tenga MENOR "Poder" acumulado.
            # Así compensamos: si un grupo ya tiene un crack, le tocará alguien normal después.
            grupos_candidatos.sort(key=lambda g: calcular_poder_grupo(g))
            mejor_grupo = grupos_candidatos[0]
            
            mejor_grupo[rol] = estudiante
            estudiante['Rol_Final'] = rol
            estudiante['Nota'] = "⭐ Experto"
        else:
            sin_asignar.append(estudiante)

    # --- FASE 2: Relleno (Backfilling) ---
    # Para los que sobraron, buscamos su segunda mejor habilidad
    for estudiante in sin_asignar:
        puntajes = {k: estudiante[k] for k in features}
        # Ordenamos sus habilidades
        orden_habilidad = sorted(puntajes, key=puntajes.get, reverse=True)
        
        asignado = False
        for habilidad in orden_habilidad:
            # Buscamos grupos con hueco en esta habilidad
            grupos_candidatos = [g for g in grupos if g[habilidad] is None]
            
            if grupos_candidatos:
                # Igual aplicamos equidad aquí
                grupos_candidatos.sort(key=lambda g: calcular_poder_grupo(g))
                target_grupo = grupos_candidatos[0]
                
                target_grupo[habilidad] = estudiante
                estudiante['Rol_Final'] = habilidad
                estudiante['Nota'] = "🔄 Cobertura"
                asignado = True
                break
        
        if not asignado:
            estudiante['Nota'] = "❌ Sin Cupo"

    return grupos

# Ejecutamos
grupos_finales = crear_grupos_equitativos(df_sorted, AMIGOS_A_JUNTAR)

# ==========================================
# 4. RESULTADOS Y MÉTRICAS
# ==========================================
datos_exportar = []
print("\n" + "="*50)
print("      GRUPOS BALANCEADOS (EQUITATIVOS)")
print("="*50)

for i, grupo in enumerate(grupos_finales):
    poder_grupo = calcular_poder_grupo(grupo)
    print(f"\n📂 GRUPO {i+1} (Poder Acumulado: {poder_grupo:.2f})")
    
    for rol in features:
        est = grupo.get(rol)
        if est:
            nombre = est['Nombres']
            puntaje = est[rol]
            nota = est.get('Nota', '')
            print(f"   [{rol}] {nombre[:25]:<25} | Nota: {puntaje:<2} ({nota})")
            
            datos_exportar.append({
                'Grupo': i+1,
                'Rol': rol,
                'Nombres': nombre,
                'Puntaje_Real': puntaje,
                'Tipo': nota,
                'Poder_Normalizado': est['Puntaje_Normalizado']
            })
        else:
            print(f"   [{rol}] --- VACÍO ---")

# Exportar
df_res = pd.DataFrame(datos_exportar)
df_res.to_excel("Grupos_Equitativos.xlsx", index=False)

# Validación de Equidad
print("\n" + "-"*30)
print("📊 ANÁLISIS DE EQUIDAD")
promedios = df_res.groupby('Grupo')['Puntaje_Real'].sum()
print(f"Puntaje Total por Grupo (Suma de notas):\n{promedios}")
print(f"\nDiferencia entre el mejor y peor grupo: {promedios.max() - promedios.min()} puntos")

⚠️ Datos en 0 detectados. Simulando puntajes aleatorios (0-10)...

      GRUPOS BALANCEADOS (EQUITATIVOS)

📂 GRUPO 1 (Poder Acumulado: 3.34)
   [A] LACOUTURE DAZA, JUAN ANDR | Nota: 10 (🤝 Amigo)
   [B] SANCHEZ GARCIA, ANDRES FE | Nota: 8  (🤝 Amigo)
   [C] PIÑEROS GARZON, CAREN NAT | Nota: 6  (⭐ Experto)
   [D] PLAZAS RIVAS, RICARDO     | Nota: 8  (⭐ Experto)

📂 GRUPO 2 (Poder Acumulado: 3.08)
   [A] BELLO ORTIZ, CARLOS DAVID | Nota: 10 (⭐ Experto)
   [B] ACERO GARCIA, SAMUEL      | Nota: 7  (⭐ Experto)
   [C] VARGAS HENAO, HAROLD STIV | Nota: 1  (🔄 Cobertura)
   [D] GIL GALLEGO, ALEJANDRO    | Nota: 9  (⭐ Experto)

📂 GRUPO 3 (Poder Acumulado: 3.05)
   [A] JEREZ RAMIREZ, MARTIN     | Nota: 10 (⭐ Experto)
   [B] LOPEZ ROMERO, VALENTINA A | Nota: 9  (⭐ Experto)
   [C] VERGARA SUAREZ, JONATAN D | Nota: 7  (⭐ Experto)
   [D] URREA LARA, DEIVID NICOLA | Nota: 4  (⭐ Experto)

📂 GRUPO 4 (Poder Acumulado: 3.32)
   [A] BELTRAN URBINA, ANDRES CA | Nota: 7  (⭐ Experto)
   [B] CARRILLO PIÑEROS, SAN